# 第一课

王晓涛<cauwxt@qq.com>


什么是PyTorch?
================

PyTorch是一个基于Python的科学计算库，它有以下特点:

- 类似于NumPy，但是它可以使用GPU
- 可以用它定义深度学习模型，可以灵活地进行深度学习模型的训练和使用

Tensors
---------------


Tensor类似与NumPy的ndarray，唯一的区别是Tensor可以在GPU上加速运算。


In [1]:
from __future__ import print_function
import torch

构造一个未初始化的5x3矩阵:

构建一个随机初始化的矩阵:

构建一个全部为0，类型为long的矩阵:

从数据直接直接构建tensor:

也可以从一个已有的tensor构建一个tensor。这些方法会重用原来tensor的特征，例如，数据类型，除非提供新的数据。

In [ ]:
                                     # result has the same size

得到tensor的形状:

<div class="alert alert-info"><h4>注意</h4><p>``torch.Size`` 返回的是一个tuple</p></div>

Operations


有很多种tensor运算。我们先介绍加法运算。



另一种着加法的写法


加法：把输出作为一个变量

in-place加法

<div class="alert alert-info"><h4>注意</h4><p>任何in-place的运算都会以``_``结尾。
    举例来说：``x.copy_(y)``, ``x.t_()``, 会改变 ``x``。</p></div>

各种类似NumPy的indexing都可以在PyTorch tensor上面使用。


Resizing: 如果你希望resize/reshape一个tensor，可以使用``torch.view``：

如果你有一个只有一个元素的tensor，使用``.item()``方法可以把里面的value变成Python数值。

**更多阅读**


  各种Tensor operations, 包括transposing, indexing, slicing,
  mathematical operations, linear algebra, random numbers在
  `<https://pytorch.org/docs/torch>`.

Numpy和Tensor之间的转化
------------

在Torch Tensor和NumPy array之间相互转化非常容易。

Torch Tensor和NumPy array会共享内存，所以改变其中一项也会改变另一项。

把Torch Tensor转变成NumPy Array


改变numpy array里面的值。

把NumPy ndarray转成Torch Tensor

所有CPU上的Tensor都支持转成numpy或者从numpy转成Tensor。

CUDA Tensors
------------

使用``.to``方法，Tensor可以被移动到别的device上。



In [2]:
# let us run this cell only if CUDA is available
# We will use ``torch.device`` objects to move tensors in and out of GPU
if torch.cuda.is_available():
    device = torch.device("cuda")          # a CUDA device object
    y = torch.ones_like(x, device=device)  # directly create a tensor on GPU
    x = x.to(device)                       # or just use strings ``.to("cuda")``
    z = x + y
    print(z)
    print(z.to("cpu", torch.double))       # ``.to`` can also change dtype together!

NameError: name 'x' is not defined


热身: 用numpy实现两层神经网络
--------------

一个全连接ReLU神经网络，一个隐藏层，没有bias。用来从x预测y，使用L2 Loss。

这一实现完全使用numpy来计算前向神经网络，loss，和反向传播。

numpy ndarray是一个普通的n维array。它不知道任何关于深度学习或者梯度(gradient)的知识，也不知道计算图(computation graph)，只是一种用来计算数学运算的数据结构。



In [3]:
import numpy as np

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random input and output data
x = np.random.randn(N, D_in)
y = np.random.randn(N, D_out)

# Randomly initialize weights
w1 = np.random.randn(D_in, H)
w2 = np.random.randn(H, D_out)

learning_rate = 1e-6
for t in range(500):
    # Forward pass: compute predicted y
    h = x.dot(w1)
    h_relu = np.maximum(h, 0)
    y_pred = h_relu.dot(w2)

    # Compute and print loss
    loss = np.square(y_pred - y).sum()
    print(t, loss)

    # Backprop to compute gradients of w1 and w2 with respect to loss
    
    # loss = (y_pred - y) ** 2
    grad_y_pred = 2.0 * (y_pred - y)
    # 
    grad_w2 = h_relu.T.dot(grad_y_pred)
    grad_h_relu = grad_y_pred.dot(w2.T)
    grad_h = grad_h_relu.copy()
    grad_h[h < 0] = 0
    grad_w1 = x.T.dot(grad_h)

    # Update weights
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

0 31260246.411749657
1 26546511.009867523
2 27201281.49060714
3 28674592.88350098
4 27767361.454228733
5 22806055.33962102
6 15660509.826203449
7 9218995.837742995
8 5062534.183504244
9 2811782.736140671
10 1693947.1213708415
11 1134061.0447826665
12 836109.3644645554
13 660223.5000345308
14 544263.390009437
15 460250.8661319723
16 395156.57484794897
17 342553.0477679961
18 298840.6364887414
19 261993.49222282256
20 230540.51706465316
21 203535.172381507
22 180230.70074317648
23 160016.86561235983
24 142424.88174835732
25 127058.46601168797
26 113579.96832661955
27 101739.18111637412
28 91320.0569265062
29 82121.72685544271
30 73975.01005427187
31 66744.38397783542
32 60309.13609170301
33 54572.37766369493
34 49446.49857428443
35 44860.793585826825
36 40751.06049754557
37 37060.831772760816
38 33743.8885846772
39 30757.794058621177
40 28065.694357289358
41 25635.56572224804
42 23438.306482855605
43 21451.405746688444
44 19651.434223201082
45 18019.17854630587
46 16539.101031462636
47 1

361 0.0014875237212221815
362 0.0014223827491915539
363 0.0013601084729342478
364 0.0013005420977857903
365 0.0012436142545274005
366 0.0011891685622898408
367 0.0011371083712561497
368 0.0010873435478532681
369 0.0010397473475525693
370 0.0009942426368644227
371 0.000950736351995439
372 0.0009091288952312883
373 0.0008693499714153713
374 0.0008313178603471722
375 0.0007949458831992317
376 0.0007601681132996963
377 0.0007269173132895689
378 0.0006951217417162386
379 0.000664715785026663
380 0.0006356463104040624
381 0.000607849411577985
382 0.000581266373378273
383 0.0005558515093470641
384 0.0005315457521936859
385 0.0005083051796237561
386 0.00048608488293530664
387 0.00046483221395580976
388 0.0004445140984700183
389 0.0004250866973767946
390 0.0004065034869699121
391 0.00038873840828832477
392 0.00037175009178639355
393 0.0003555011151679184
394 0.0003399685497143403
395 0.00032511326147982486
396 0.0003109055527556519
397 0.00029732409869727866
398 0.0002843332679776095
399 0.0002


PyTorch: Tensors
----------------

这次我们使用PyTorch tensors来创建前向神经网络，计算损失，以及反向传播。

一个PyTorch Tensor很像一个numpy的ndarray。但是它和numpy ndarray最大的区别是，PyTorch Tensor可以在CPU或者GPU上运算。如果想要在GPU上运算，就需要把Tensor换成cuda类型。


In [4]:
import torch


dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0") # Uncomment this to run on GPU

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random input and output data
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# Randomly initialize weights
w1 = torch.randn(D_in, H, device=device, dtype=dtype)
w2 = torch.randn(H, D_out, device=device, dtype=dtype)

learning_rate = 1e-6
for t in range(500):
    # Forward pass: compute predicted y
    h = x.mm(w1)
    h_relu = h.clamp(min=0)
    y_pred = h_relu.mm(w2)

    # Compute and print loss
    loss = (y_pred - y).pow(2).sum().item()
    print(t, loss)

    # Backprop to compute gradients of w1 and w2 with respect to loss
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.t().mm(grad_y_pred)
    grad_h_relu = grad_y_pred.mm(w2.t())
    grad_h = grad_h_relu.clone()
    grad_h[h < 0] = 0
    grad_w1 = x.t().mm(grad_h)

    # Update weights using gradient descent
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

0 39992300.0
1 41185612.0
2 44618008.0
3 40547396.0
4 27778012.0
5 14144622.0
6 6349229.5
7 3139666.5
8 1925899.875
9 1400285.875
10 1112990.375
11 920752.25
12 776336.1875
13 661717.0625
14 568255.25
15 491163.5
16 426773.53125
17 372520.8125
18 326517.5625
19 287334.65625
20 253744.28125
21 224813.9375
22 199777.34375
23 178032.875
24 159048.6875
25 142435.765625
26 127861.78125
27 115040.03125
28 103702.25
29 93660.46875
30 84745.96875
31 76813.8359375
32 69731.265625
33 63396.1015625
34 57718.7109375
35 52619.63671875
36 48029.32421875
37 43890.35546875
38 40156.25390625
39 36779.6953125
40 33725.578125
41 30957.22265625
42 28444.310546875
43 26159.615234375
44 24079.646484375
45 22183.228515625
46 20452.427734375
47 18870.9921875
48 17424.841796875
49 16101.4462890625
50 14888.76953125
51 13776.92578125
52 12756.94140625
53 11820.36328125
54 10959.21875
55 10167.6943359375
56 9438.7607421875
57 8767.15625
58 8148.1240234375
59 7577.052734375
60 7050.74951171875
61 6565.3125
62 611

378 0.0011588472407311201
379 0.0011182907037436962
380 0.0010792964603751898
381 0.0010425273794680834
382 0.001006508362479508
383 0.0009704729309305549
384 0.000938737764954567
385 0.0009080087766051292
386 0.0008777950424700975
387 0.0008497098460793495
388 0.000819699838757515
389 0.0007925801910459995
390 0.0007680226117372513
391 0.0007434476865455508
392 0.0007187966257333755
393 0.0006966863293200731
394 0.0006743226549588144
395 0.0006539690075442195
396 0.0006335899233818054
397 0.0006133493734523654
398 0.0005942268762737513
399 0.0005761347711086273
400 0.000558895873837173
401 0.0005417743232101202
402 0.000525482464581728
403 0.0005094424122944474
404 0.00049453234532848
405 0.0004805568605661392
406 0.00046678134822286665
407 0.00045341558870859444
408 0.0004407507076393813
409 0.0004279001150280237
410 0.0004156938230153173
411 0.0004044497909490019
412 0.00039300232310779393
413 0.0003819395788013935
414 0.0003711057361215353
415 0.00036195587017573416
416 0.000351432

简单的autograd

In [5]:
# Create tensors.
x = torch.tensor(1., requires_grad=True)
w = torch.tensor(2., requires_grad=True)
b = torch.tensor(3., requires_grad=True)

# Build a computational graph.
y = w * x + b    # y = 2 * x + 3

# Compute gradients.
y.backward()

# Print out the gradients.
print(x.grad)    # x.grad = 2 
print(w.grad)    # w.grad = 1 
print(b.grad)    # b.grad = 1 

tensor(2.)
tensor(1.)
tensor(1.)



PyTorch: Tensor和autograd
-------------------------------

PyTorch的一个重要功能就是autograd，也就是说只要定义了forward pass(前向神经网络)，计算了loss之后，PyTorch可以自动求导计算模型所有参数的梯度。

一个PyTorch的Tensor表示计算图中的一个节点。如果``x``是一个Tensor并且``x.requires_grad=True``那么``x.grad``是另一个储存着``x``当前梯度(相对于一个scalar，常常是loss)的向量。


In [6]:
import torch

dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0") # Uncomment this to run on GPU

# N 是 batch size; D_in 是 input dimension;
# H 是 hidden dimension; D_out 是 output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# 创建随机的Tensor来保存输入和输出
# 设定requires_grad=False表示在反向传播的时候我们不需要计算gradient
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# 创建随机的Tensor和权重。
# 设置requires_grad=True表示我们希望反向传播的时候计算Tensor的gradient
w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(500):
    # 前向传播:通过Tensor预测y；这个和普通的神经网络的前向传播没有任何不同，
    # 但是我们不需要保存网络的中间运算结果，因为我们不需要手动计算反向传播。
    y_pred = x.mm(w1).clamp(min=0).mm(w2)

    # 通过前向传播计算loss
    # loss是一个形状为(1，)的Tensor
    # loss.item()可以给我们返回一个loss的scalar
    loss = (y_pred - y).pow(2).sum()
    print(t, loss.item())

    # PyTorch给我们提供了autograd的方法做反向传播。如果一个Tensor的requires_grad=True，
    # backward会自动计算loss相对于每个Tensor的gradient。在backward之后，
    # w1.grad和w2.grad会包含两个loss相对于两个Tensor的gradient信息。
    loss.backward()

    # 我们可以手动做gradient descent(后面我们会介绍自动的方法)。
    # 用torch.no_grad()包含以下statements，因为w1和w2都是requires_grad=True，
    # 但是在更新weights之后我们并不需要再做autograd。
    # 另一种方法是在weight.data和weight.grad.data上做操作，这样就不会对grad产生影响。
    # tensor.data会我们一个tensor，这个tensor和原来的tensor指向相同的内存空间，
    # 但是不会记录计算图的历史。
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad

        # Manually zero the gradients after updating weights
        w1.grad.zero_()
        w2.grad.zero_()

0 27484794.0
1 25186148.0
2 26499672.0
3 27811656.0
4 26498382.0
5 21366974.0
6 14516730.0
7 8542893.0
8 4735615.0
9 2664450.5
10 1622764.875
11 1092537.75
12 806148.5
13 635412.125
14 522389.5
15 440550.6875
16 377392.0625
17 326495.46875
18 284465.375
19 249142.46875
20 219169.796875
21 193523.078125
22 171425.4375
23 152314.140625
24 135731.796875
25 121290.828125
26 108650.5546875
27 97553.484375
28 87782.5546875
29 79149.46875
30 71507.9921875
31 64727.0546875
32 58689.7890625
33 53316.9296875
34 48510.96484375
35 44203.50390625
36 40340.703125
37 36863.23046875
38 33729.79296875
39 30905.220703125
40 28350.140625
41 26035.998046875
42 23934.515625
43 22025.091796875
44 20288.197265625
45 18705.76171875
46 17262.4609375
47 15943.8671875
48 14737.8857421875
49 13633.216796875
50 12620.9921875
51 11692.1796875
52 10837.201171875
53 10052.171875
54 9329.849609375
55 8664.591796875
56 8051.71533203125
57 7486.51025390625
58 6965.64306640625
59 6486.21923828125
60 6043.001953125
61 563


PyTorch: nn
-----------


这次我们使用PyTorch中nn这个库来构建网络。
用PyTorch autograd来构建计算图和计算gradients，
然后PyTorch会帮我们自动计算gradient。




In [7]:
import torch

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold inputs and outputs
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# Use the nn package to define our model as a sequence of layers. nn.Sequential
# is a Module which contains other Modules, and applies them in sequence to
# produce its output. Each Linear Module computes output from input using a
# linear function, and holds internal Tensors for its weight and bias.
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)

# The nn package also contains definitions of popular loss functions; in this
# case we will use Mean Squared Error (MSE) as our loss function.
loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-4
for t in range(500):
    # Forward pass: compute predicted y by passing x to the model. Module objects
    # override the __call__ operator so you can call them like functions. When
    # doing so you pass a Tensor of input data to the Module and it produces
    # a Tensor of output data.
    y_pred = model(x)

    # Compute and print loss. We pass Tensors containing the predicted and true
    # values of y, and the loss function returns a Tensor containing the
    # loss.
    loss = loss_fn(y_pred, y)
    print(t, loss.item())

    # Zero the gradients before running the backward pass.
    model.zero_grad()

    # Backward pass: compute gradient of the loss with respect to all the learnable
    # parameters of the model. Internally, the parameters of each Module are stored
    # in Tensors with requires_grad=True, so this call will compute gradients for
    # all learnable parameters in the model.
    loss.backward()

    # Update the weights using gradient descent. Each parameter is a Tensor, so
    # we can access its gradients like we did before.
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad

0 620.5733642578125
1 571.293701171875
2 529.0409545898438
3 492.3656311035156
4 459.8135986328125
5 430.5155029296875
6 404.1044006347656
7 380.03692626953125
8 357.8116455078125
9 337.3223571777344
10 318.09051513671875
11 300.06781005859375
12 283.1866455078125
13 267.205078125
14 252.0729522705078
15 237.79379272460938
16 224.301513671875
17 211.51817321777344
18 199.46197509765625
19 188.07485961914062
20 177.29080200195312
21 167.09542846679688
22 157.40750122070312
23 148.26039123535156
24 139.60470581054688
25 131.42649841308594
26 123.69979858398438
27 116.38845825195312
28 109.4806137084961
29 102.9622573852539
30 96.81407165527344
31 91.02054595947266
32 85.57273864746094
33 80.44019317626953
34 75.61373901367188
35 71.07894134521484
36 66.81837463378906
37 62.82241439819336
38 59.07289505004883
39 55.556522369384766
40 52.253807067871094
41 49.15754699707031
42 46.25043869018555
43 43.52094268798828
44 40.962318420410156
45 38.563846588134766
46 36.30937576293945
47 34.2000

401 7.626089791301638e-05
402 7.415669824695215e-05
403 7.211537740658969e-05
404 7.013075082795694e-05
405 6.82039390085265e-05
406 6.633048178628087e-05
407 6.450699584092945e-05
408 6.273386679822579e-05
409 6.101321923779324e-05
410 5.933863576501608e-05
411 5.771401629317552e-05
412 5.613020766759291e-05
413 5.4596261179540306e-05
414 5.310078267939389e-05
415 5.164902177057229e-05
416 5.023710764362477e-05
417 4.886589522357099e-05
418 4.75314081995748e-05
419 4.623486165655777e-05
420 4.497456757235341e-05
421 4.3751027988037094e-05
422 4.255849853507243e-05
423 4.14010755775962e-05
424 4.0272097976412624e-05
425 3.9181788451969624e-05
426 3.811478745774366e-05
427 3.707895666593686e-05
428 3.607549297157675e-05
429 3.5093129554297775e-05
430 3.414317689021118e-05
431 3.321848635096103e-05
432 3.231843948015012e-05
433 3.144356742268428e-05
434 3.059373557334766e-05
435 2.976709947688505e-05
436 2.896072510338854e-05
437 2.8184394977870397e-05
438 2.7421152481110767e-05
439 2.66


PyTorch: optim
--------------

这一次我们不再手动更新模型的weights,而是使用optim这个包来帮助我们更新参数。
optim这个package提供了各种不同的模型优化方法，包括SGD+momentum, RMSProp, Adam等等。


In [8]:
import torch

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold inputs and outputs
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# Use the nn package to define our model and loss function.
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)
loss_fn = torch.nn.MSELoss(reduction='sum')

# Use the optim package to define an Optimizer that will update the weights of
# the model for us. Here we will use Adam; the optim package contains many other
# optimization algoriths. The first argument to the Adam constructor tells the
# optimizer which Tensors it should update.
learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for t in range(500):
    # Forward pass: compute predicted y by passing x to the model.
    y_pred = model(x)

    # Compute and print loss.
    loss = loss_fn(y_pred, y)
    print(t, loss.item())

    # Before the backward pass, use the optimizer object to zero all of the
    # gradients for the variables it will update (which are the learnable
    # weights of the model). This is because by default, gradients are
    # accumulated in buffers( i.e, not overwritten) whenever .backward()
    # is called. Checkout docs of torch.autograd.backward for more details.
    optimizer.zero_grad()

    # Backward pass: compute gradient of the loss with respect to model
    # parameters
    loss.backward()

    # Calling the step function on an Optimizer makes an update to its
    # parameters
    optimizer.step()

0 619.4041748046875
1 603.1110229492188
2 587.2695922851562
3 571.850830078125
4 556.8591918945312
5 542.2738647460938
6 528.0520629882812
7 514.189208984375
8 500.81378173828125
9 487.809326171875
10 475.2987060546875
11 463.1747741699219
12 451.4615783691406
13 440.12237548828125
14 429.0684814453125
15 418.3021545410156
16 407.7438049316406
17 397.4444580078125
18 387.4076843261719
19 377.6274719238281
20 368.0901184082031
21 358.7952880859375
22 349.7372131347656
23 340.906005859375
24 332.28973388671875
25 323.9122009277344
26 315.7377014160156
27 307.7489929199219
28 299.9654235839844
29 292.38507080078125
30 284.9714660644531
31 277.7470703125
32 270.651123046875
33 263.6981201171875
34 256.8956604003906
35 250.253662109375
36 243.7548370361328
37 237.4071807861328
38 231.18507385253906
39 225.09231567382812
40 219.14073181152344
41 213.31344604492188
42 207.59295654296875
43 201.98304748535156
44 196.4952392578125
45 191.1175994873047
46 185.8562469482422
47 180.7108154296875
4

426 1.4392276259656e-09
427 1.3475736082568801e-09
428 1.2555183559470606e-09
429 1.1850953551828525e-09
430 1.1078031825206835e-09
431 1.0416733031703984e-09
432 9.806724321492766e-10
433 9.197524963866499e-10
434 8.712984778114219e-10
435 8.14494693912593e-10
436 7.637628862688928e-10
437 7.211815589158732e-10
438 6.794754203731657e-10
439 6.457345214094801e-10
440 6.058115675777742e-10
441 5.750903642187666e-10
442 5.411129877508358e-10
443 5.140519121482612e-10
444 4.859382896071907e-10
445 4.586113711013695e-10
446 4.3512876635176667e-10
447 4.1456915678139694e-10
448 3.8931696755284406e-10
449 3.67793517863646e-10
450 3.519370905813446e-10
451 3.337908005551782e-10
452 3.141437110443235e-10
453 3.0254215799274675e-10
454 2.879710359060539e-10
455 2.7465860141795417e-10
456 2.606502791380194e-10
457 2.512381136465791e-10
458 2.386495168149594e-10
459 2.2848578584699908e-10
460 2.1820956153106863e-10
461 2.0644329301600095e-10
462 2.0044046977751862e-10
463 1.9204671186656697e-10
4


PyTorch: 自定义 nn Modules
--------------------------

我们可以定义一个模型，这个模型继承自nn.Module类。如果需要定义一个比Sequential模型更加复杂的模型，就需要定义nn.Module模型。



In [9]:
import torch


class TwoLayerNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        In the constructor we instantiate two nn.Linear modules and assign them as
        member variables.
        """
        super(TwoLayerNet, self).__init__()
        self.linear1 = torch.nn.Linear(D_in, H)
        self.linear2 = torch.nn.Linear(H, D_out)

    def forward(self, x):
        """
        In the forward function we accept a Tensor of input data and we must return
        a Tensor of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Tensors.
        """
        h_relu = self.linear1(x).clamp(min=0)
        y_pred = self.linear2(h_relu)
        return y_pred


# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold inputs and outputs
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# Construct our model by instantiating the class defined above
model = TwoLayerNet(D_in, H, D_out)

# Construct our loss function and an Optimizer. The call to model.parameters()
# in the SGD constructor will contain the learnable parameters of the two
# nn.Linear modules which are members of the model.
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)
for t in range(500):
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(x)

    # Compute and print loss
    loss = criterion(y_pred, y)
    print(t, loss.item())

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

0 687.2293701171875
1 634.19189453125
2 588.5949096679688
3 548.7039184570312
4 513.4244384765625
5 482.0264587402344
6 453.6788330078125
7 427.80926513671875
8 404.1365661621094
9 382.3700866699219
10 362.061279296875
11 342.96490478515625
12 324.990478515625
13 307.9693603515625
14 291.95574951171875
15 276.76947021484375
16 262.1172790527344
17 248.11285400390625
18 234.79629516601562
19 222.14688110351562
20 210.0814208984375
21 198.62844848632812
22 187.7060089111328
23 177.27169799804688
24 167.33160400390625
25 157.83274841308594
26 148.77952575683594
27 140.1869659423828
28 131.96580505371094
29 124.1637954711914
30 116.76571655273438
31 109.747802734375
32 103.10079956054688
33 96.81016540527344
34 90.86135864257812
35 85.23873138427734
36 79.93568420410156
37 74.9471435546875
38 70.24595642089844
39 65.81832885742188
40 61.655059814453125
41 57.732093811035156
42 54.04985809326172
43 50.59822463989258
44 47.36359786987305
45 44.338504791259766
46 41.50608444213867
47 38.84733

444 4.116254785913043e-06
445 3.993028713011881e-06
446 3.87336285712081e-06
447 3.7574150155705865e-06
448 3.644649268608191e-06
449 3.5359253161004744e-06
450 3.429631988183246e-06
451 3.326909336465178e-06
452 3.2277207537845243e-06
453 3.130404820694821e-06
454 3.0369674277608283e-06
455 2.9461200483638095e-06
456 2.8588967779796803e-06
457 2.773228743535583e-06
458 2.689662778720958e-06
459 2.6094221539096907e-06
460 2.5317474410258e-06
461 2.456156835251022e-06
462 2.382673528700252e-06
463 2.311317075509578e-06
464 2.2424972030421486e-06
465 2.175990857722354e-06
466 2.1103144263179274e-06
467 2.0477159523579758e-06
468 1.9866299680870725e-06
469 1.9275184968137182e-06
470 1.8701000499277143e-06
471 1.814253209886374e-06
472 1.7603261994736386e-06
473 1.7079676126741106e-06
474 1.6568108094361378e-06
475 1.607604417586117e-06
476 1.5597463516314747e-06
477 1.513082111159747e-06
478 1.4682266282761702e-06
479 1.42461726682086e-06
480 1.3822332221025135e-06
481 1.341399752163852e-

# FizzBuzz

FizzBuzz是一个简单的小游戏。游戏规则如下：从1开始往上数数，当遇到3的倍数的时候，说fizz，当遇到5的倍数，说buzz，当遇到15的倍数，就说fizzbuzz，其他情况下则正常数数。

我们可以写一个简单的小程序来决定要返回正常数值还是fizz, buzz 或者 fizzbuzz。

In [10]:
# One-hot encode the desired outputs: [number, "fizz", "buzz", "fizzbuzz"]
def fizz_buzz_encode(i):
    if   i % 15 == 0: return 3
    elif i % 5  == 0: return 2
    elif i % 3  == 0: return 1
    else:             return 0
    
def fizz_buzz_decode(i, prediction):
    return [str(i), "fizz", "buzz", "fizzbuzz"][prediction]

print(fizz_buzz_decode(1, fizz_buzz_encode(1)))
print(fizz_buzz_decode(2, fizz_buzz_encode(2)))
print(fizz_buzz_decode(5, fizz_buzz_encode(5)))
print(fizz_buzz_decode(12, fizz_buzz_encode(12)))
print(fizz_buzz_decode(15, fizz_buzz_encode(15)))

1
2
buzz
fizz
fizzbuzz


我们首先定义模型的输入与输出(训练数据)

In [11]:
import numpy as np
import torch

NUM_DIGITS = 10

# Represent each input by an array of its binary digits.
def binary_encode(i, num_digits):
    return np.array([i >> d & 1 for d in range(num_digits)])

trX = torch.Tensor([binary_encode(i, NUM_DIGITS) for i in range(101, 2 ** NUM_DIGITS)])
trY = torch.LongTensor([fizz_buzz_encode(i) for i in range(101, 2 ** NUM_DIGITS)])

然后我们用PyTorch定义模型

In [14]:
# Define the model
NUM_HIDDEN = 100
model = torch.nn.Sequential(
    torch.nn.Linear(NUM_DIGITS, NUM_HIDDEN),
    torch.nn.ReLU(),
    torch.nn.Linear(NUM_HIDDEN, 4)
)

- 为了让我们的模型学会FizzBuzz这个游戏，我们需要定义一个损失函数，和一个优化算法。
- 这个优化算法会不断优化（降低）损失函数，使得模型的在该任务上取得尽可能低的损失值。
- 损失值低往往表示我们的模型表现好，损失值高表示我们的模型表现差。
- 由于FizzBuzz游戏本质上是一个分类问题，我们选用Cross Entropyy Loss函数。
- 优化函数我们选用Stochastic Gradient Descent。

In [15]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.05)

以下是模型的训练代码

In [16]:
# Start training it
BATCH_SIZE = 128
for epoch in range(10000):
    for start in range(0, len(trX), BATCH_SIZE):
        end = start + BATCH_SIZE
        batchX = trX[start:end]
        batchY = trY[start:end]

        y_pred = model(batchX)
        loss = loss_fn(y_pred, batchY)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Find loss on training data
    loss = loss_fn(model(trX), trY).item()
    print('Epoch:', epoch, 'Loss:', loss)

Epoch: 0 Loss: 1.1752445697784424
Epoch: 1 Loss: 1.1527748107910156
Epoch: 2 Loss: 1.1471346616744995
Epoch: 3 Loss: 1.1449143886566162
Epoch: 4 Loss: 1.1437774896621704
Epoch: 5 Loss: 1.143070936203003
Epoch: 6 Loss: 1.1425672769546509
Epoch: 7 Loss: 1.1421717405319214
Epoch: 8 Loss: 1.1418371200561523
Epoch: 9 Loss: 1.1415421962738037
Epoch: 10 Loss: 1.1412750482559204
Epoch: 11 Loss: 1.141027808189392
Epoch: 12 Loss: 1.140796184539795
Epoch: 13 Loss: 1.1405755281448364
Epoch: 14 Loss: 1.1403676271438599
Epoch: 15 Loss: 1.1401653289794922
Epoch: 16 Loss: 1.13997220993042
Epoch: 17 Loss: 1.139786958694458
Epoch: 18 Loss: 1.139608383178711
Epoch: 19 Loss: 1.1394349336624146
Epoch: 20 Loss: 1.1392674446105957
Epoch: 21 Loss: 1.1391061544418335
Epoch: 22 Loss: 1.1389487981796265
Epoch: 23 Loss: 1.1387975215911865
Epoch: 24 Loss: 1.1386489868164062
Epoch: 25 Loss: 1.138502597808838
Epoch: 26 Loss: 1.138362169265747
Epoch: 27 Loss: 1.1382249593734741
Epoch: 28 Loss: 1.1380912065505981
Epoc

Epoch: 233 Loss: 1.0845177173614502
Epoch: 234 Loss: 1.084859848022461
Epoch: 235 Loss: 1.0831843614578247
Epoch: 236 Loss: 1.0825752019882202
Epoch: 237 Loss: 1.0834254026412964
Epoch: 238 Loss: 1.0817445516586304
Epoch: 239 Loss: 1.082134485244751
Epoch: 240 Loss: 1.0805970430374146
Epoch: 241 Loss: 1.0810049772262573
Epoch: 242 Loss: 1.0793569087982178
Epoch: 243 Loss: 1.0797879695892334
Epoch: 244 Loss: 1.0793440341949463
Epoch: 245 Loss: 1.0779162645339966
Epoch: 246 Loss: 1.078298807144165
Epoch: 247 Loss: 1.0769002437591553
Epoch: 248 Loss: 1.0768775939941406
Epoch: 249 Loss: 1.075770378112793
Epoch: 250 Loss: 1.0745978355407715
Epoch: 251 Loss: 1.0754570960998535
Epoch: 252 Loss: 1.0746948719024658
Epoch: 253 Loss: 1.0732218027114868
Epoch: 254 Loss: 1.0733839273452759
Epoch: 255 Loss: 1.0719608068466187
Epoch: 256 Loss: 1.072079062461853
Epoch: 257 Loss: 1.071616530418396
Epoch: 258 Loss: 1.0706380605697632
Epoch: 259 Loss: 1.0706173181533813
Epoch: 260 Loss: 1.069990634918213

Epoch: 464 Loss: 0.9357263445854187
Epoch: 465 Loss: 0.9347310662269592
Epoch: 466 Loss: 0.9330903887748718
Epoch: 467 Loss: 0.9341756701469421
Epoch: 468 Loss: 0.9320069551467896
Epoch: 469 Loss: 0.930217444896698
Epoch: 470 Loss: 0.9315487742424011
Epoch: 471 Loss: 0.928566575050354
Epoch: 472 Loss: 0.9311053156852722
Epoch: 473 Loss: 0.9272609353065491
Epoch: 474 Loss: 0.9271014928817749
Epoch: 475 Loss: 0.9259443283081055
Epoch: 476 Loss: 0.925812840461731
Epoch: 477 Loss: 0.924372673034668
Epoch: 478 Loss: 0.9241588115692139
Epoch: 479 Loss: 0.9230336546897888
Epoch: 480 Loss: 0.9242668151855469
Epoch: 481 Loss: 0.9208613634109497
Epoch: 482 Loss: 0.9199551939964294
Epoch: 483 Loss: 0.9210118651390076
Epoch: 484 Loss: 0.9187893867492676
Epoch: 485 Loss: 0.9198324084281921
Epoch: 486 Loss: 0.916894257068634
Epoch: 487 Loss: 0.9160569310188293
Epoch: 488 Loss: 0.9162460565567017
Epoch: 489 Loss: 0.9155914187431335
Epoch: 490 Loss: 0.9149048924446106
Epoch: 491 Loss: 0.91507148742675

Epoch: 693 Loss: 0.7160546779632568
Epoch: 694 Loss: 0.7149877548217773
Epoch: 695 Loss: 0.7129070162773132
Epoch: 696 Loss: 0.7130376696586609
Epoch: 697 Loss: 0.7087226510047913
Epoch: 698 Loss: 0.708712637424469
Epoch: 699 Loss: 0.7086120843887329
Epoch: 700 Loss: 0.7084261178970337
Epoch: 701 Loss: 0.7074194550514221
Epoch: 702 Loss: 0.7073612809181213
Epoch: 703 Loss: 0.7039808034896851
Epoch: 704 Loss: 0.7026575207710266
Epoch: 705 Loss: 0.7006410956382751
Epoch: 706 Loss: 0.7013440132141113
Epoch: 707 Loss: 0.7000851035118103
Epoch: 708 Loss: 0.698195219039917
Epoch: 709 Loss: 0.6985461711883545
Epoch: 710 Loss: 0.6969631314277649
Epoch: 711 Loss: 0.6980965733528137
Epoch: 712 Loss: 0.6946825385093689
Epoch: 713 Loss: 0.6947281360626221
Epoch: 714 Loss: 0.6928823590278625
Epoch: 715 Loss: 0.6917062401771545
Epoch: 716 Loss: 0.6893540620803833
Epoch: 717 Loss: 0.6890638470649719
Epoch: 718 Loss: 0.6883246898651123
Epoch: 719 Loss: 0.6886910796165466
Epoch: 720 Loss: 0.68699157238

Epoch: 923 Loss: 0.4852830469608307
Epoch: 924 Loss: 0.48484867811203003
Epoch: 925 Loss: 0.4828505516052246
Epoch: 926 Loss: 0.4830262362957001
Epoch: 927 Loss: 0.48098206520080566
Epoch: 928 Loss: 0.48141157627105713
Epoch: 929 Loss: 0.48000141978263855
Epoch: 930 Loss: 0.4791625738143921
Epoch: 931 Loss: 0.47828519344329834
Epoch: 932 Loss: 0.4773813486099243
Epoch: 933 Loss: 0.4771408140659332
Epoch: 934 Loss: 0.47639819979667664
Epoch: 935 Loss: 0.47618934512138367
Epoch: 936 Loss: 0.47437945008277893
Epoch: 937 Loss: 0.47371840476989746
Epoch: 938 Loss: 0.47258293628692627
Epoch: 939 Loss: 0.47198134660720825
Epoch: 940 Loss: 0.4710565507411957
Epoch: 941 Loss: 0.4705093502998352
Epoch: 942 Loss: 0.46970996260643005
Epoch: 943 Loss: 0.4700033366680145
Epoch: 944 Loss: 0.46833837032318115
Epoch: 945 Loss: 0.4671805202960968
Epoch: 946 Loss: 0.46617591381073
Epoch: 947 Loss: 0.466474711894989
Epoch: 948 Loss: 0.46504345536231995
Epoch: 949 Loss: 0.4640341103076935
Epoch: 950 Loss: 

...

Epoch: 8378 Loss: 0.008947393856942654
Epoch: 8379 Loss: 0.00894198939204216
Epoch: 8380 Loss: 0.008943016640841961
Epoch: 8381 Loss: 0.008941993117332458
Epoch: 8382 Loss: 0.00893646851181984
Epoch: 8383 Loss: 0.008936138823628426
Epoch: 8384 Loss: 0.008935471065342426
Epoch: 8385 Loss: 0.008932594209909439
Epoch: 8386 Loss: 0.00892980583012104
Epoch: 8387 Loss: 0.00892753154039383
Epoch: 8388 Loss: 0.008926843293011189
Epoch: 8389 Loss: 0.008925186470150948
Epoch: 8390 Loss: 0.008923332206904888
Epoch: 8391 Loss: 0.00892181321978569
Epoch: 8392 Loss: 0.008921463042497635
Epoch: 8393 Loss: 0.00891687348484993
Epoch: 8394 Loss: 0.008915513753890991
Epoch: 8395 Loss: 0.00891310814768076
Epoch: 8396 Loss: 0.008914324454963207
Epoch: 8397 Loss: 0.008911027573049068
Epoch: 8398 Loss: 0.00890566036105156
Epoch: 8399 Loss: 0.008907795883715153
Epoch: 8400 Loss: 0.008906764909625053
Epoch: 8401 Loss: 0.008900690823793411
Epoch: 8402 Loss: 0.008900542743504047
Epoch: 8403 Loss: 0.0089021064341

Epoch: 8604 Loss: 0.008544682525098324
Epoch: 8605 Loss: 0.008544684387743473
Epoch: 8606 Loss: 0.008544040843844414
Epoch: 8607 Loss: 0.008543560281395912
Epoch: 8608 Loss: 0.008538462221622467
Epoch: 8609 Loss: 0.008538593538105488
Epoch: 8610 Loss: 0.008539178408682346
Epoch: 8611 Loss: 0.00853512343019247
Epoch: 8612 Loss: 0.008533341810107231
Epoch: 8613 Loss: 0.008533624932169914
Epoch: 8614 Loss: 0.008529353886842728
Epoch: 8615 Loss: 0.008528451435267925
Epoch: 8616 Loss: 0.008528130128979683
Epoch: 8617 Loss: 0.008524374105036259
Epoch: 8618 Loss: 0.008525579236447811
Epoch: 8619 Loss: 0.008523046970367432
Epoch: 8620 Loss: 0.008521104231476784
Epoch: 8621 Loss: 0.008516672067344189
Epoch: 8622 Loss: 0.008517042733728886
Epoch: 8623 Loss: 0.00851520150899887
Epoch: 8624 Loss: 0.008513791486620903
Epoch: 8625 Loss: 0.008513854816555977
Epoch: 8626 Loss: 0.008505510166287422
Epoch: 8627 Loss: 0.008509596809744835
Epoch: 8628 Loss: 0.008506360463798046
Epoch: 8629 Loss: 0.0085081

Epoch: 8837 Loss: 0.008170551620423794
Epoch: 8838 Loss: 0.008167793974280357
Epoch: 8839 Loss: 0.008164811879396439
Epoch: 8840 Loss: 0.008165650069713593
Epoch: 8841 Loss: 0.00816398486495018
Epoch: 8842 Loss: 0.008159701712429523
Epoch: 8843 Loss: 0.008157346397638321
Epoch: 8844 Loss: 0.008158671669661999
Epoch: 8845 Loss: 0.008155964314937592
Epoch: 8846 Loss: 0.008154930546879768
Epoch: 8847 Loss: 0.008151599206030369
Epoch: 8848 Loss: 0.008153031580150127
Epoch: 8849 Loss: 0.008149291388690472
Epoch: 8850 Loss: 0.008150257170200348
Epoch: 8851 Loss: 0.008146768435835838
Epoch: 8852 Loss: 0.0081437723711133
Epoch: 8853 Loss: 0.008142848499119282
Epoch: 8854 Loss: 0.008142662234604359
Epoch: 8855 Loss: 0.008140676654875278
Epoch: 8856 Loss: 0.008138191886246204
Epoch: 8857 Loss: 0.008138657547533512
Epoch: 8858 Loss: 0.008137345314025879
Epoch: 8859 Loss: 0.008134963922202587
Epoch: 8860 Loss: 0.008132047951221466
Epoch: 8861 Loss: 0.00813095923513174
Epoch: 8862 Loss: 0.008130721

Epoch: 9066 Loss: 0.007825756445527077
Epoch: 9067 Loss: 0.00782162044197321
Epoch: 9068 Loss: 0.00782177783548832
Epoch: 9069 Loss: 0.007820550352334976
Epoch: 9070 Loss: 0.007817714475095272
Epoch: 9071 Loss: 0.007818377576768398
Epoch: 9072 Loss: 0.007815380580723286
Epoch: 9073 Loss: 0.00781499594449997
Epoch: 9074 Loss: 0.007813607342541218
Epoch: 9075 Loss: 0.007809776347130537
Epoch: 9076 Loss: 0.0078086755238473415
Epoch: 9077 Loss: 0.007808041293174028
Epoch: 9078 Loss: 0.007807029411196709
Epoch: 9079 Loss: 0.007805798668414354
Epoch: 9080 Loss: 0.007805380504578352


KeyboardInterrupt: 

最后我们用训练好的模型尝试在1到100这些数字上玩FizzBuzz游戏

In [17]:
# Output now
testX = torch.Tensor([binary_encode(i, NUM_DIGITS) for i in range(1, 101)])
with torch.no_grad():
    testY = model(testX)
predictions = zip(range(1, 101), list(testY.max(1)[1].data.tolist()))

print([fizz_buzz_decode(i, x) for (i, x) in predictions])

['1', '2', 'fizz', '4', 'buzz', 'fizz', '7', '8', 'fizz', 'buzz', '11', 'fizz', '13', '14', 'fizzbuzz', '16', '17', 'fizz', '19', 'buzz', 'fizz', '22', '23', 'fizz', 'buzz', '26', 'fizz', '28', '29', 'fizzbuzz', '31', '32', 'fizz', '34', 'buzz', 'fizz', '37', '38', 'fizz', 'buzz', '41', 'fizz', '43', '44', 'fizzbuzz', '46', '47', 'fizz', '49', 'buzz', 'fizz', '52', '53', 'fizz', 'buzz', '56', 'fizz', '58', '59', 'fizzbuzz', '61', '62', 'fizz', '64', 'buzz', 'fizz', '67', '68', 'fizz', 'buzz', '71', 'fizz', '73', '74', 'fizzbuzz', '76', '77', 'fizz', '79', 'buzz', 'fizz', '82', '83', 'fizz', 'buzz', '86', '87', '88', '89', 'fizzbuzz', '91', '92', '93', '94', 'buzz', 'fizz', '97', '98', 'fizz', 'buzz']


In [18]:
print(np.sum(testY.max(1)[1].numpy() == np.array([fizz_buzz_encode(i) for i in range(1,101)])))
testY.max(1)[1].numpy() == np.array([fizz_buzz_encode(i) for i in range(1,101)])

98


array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True, False,  True,  True,  True,  True,  True,  True,
        True])